In [2]:
import shutil
import os 
import re
import json

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import albumentations as A

import torch
from torch.utils.data import DataLoader

from torchvision.transforms import ToPILImage
tensor2pil = ToPILImage()

from tools.dataset_tools import *
from tools.model_tools import *

import tqdm

/home/raph/fast_mem/anaconda3/envs/ImgSegment/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
root_path = "outputs"
exp_paths = [
    os.path.join("2023-05-23", "04-30-01"),
    os.path.join("2023-05-24", "16-24-53")
]

In [4]:
target_metric = "soft_jaccard"
metric_value = 0.85


In [5]:
dataset_cfg = {
    "device": "cuda",
    "path": "../datasets/datasets.json",
    "max_subset_size": -1,
    
    "bgr_trfm": [],
    "fgr_trfm": [],
    "trfm": [],
    "preproc": [],
}

with open(dataset_cfg["path"]) as f:
    file_dict = json.load(f)

data_path = os.path.join(os.path.dirname(dataset_cfg["path"]), file_dict["root_dir"])

In [6]:
model_cfg = {
    "name": "mini_u2net",
    "device": "cuda:0",
    "load_pretrained": True,
    "pretrained_path": "",
    
    "freeze_encoder": [False, False, False, False, False, False],
    "freeze_decoder": [False, False, False, False, False],
    "freeze_side": [False, False, False, False, False, False],

    "out_ch": 1
}

In [ ]:
def illustrate(model, loader, save_path):
    model.eval()
    with torch.no_grad():
        for dir_names, img_names, img_batch, mask_batch in tqdm.tqdm(loader):
            predicted_batch = model.inference(img_batch)[0]

            masked_batch = img_batch * predicted_batch
            for i, masked_tensor in enumerate(masked_batch):
                masked_pil = tensor2pil(masked_tensor)
                masked_pil.save(os.path.join(save_path, dir_names[i], img_names[i] + ".jpg"))
                

In [7]:
save_dir = "bad_pictures" 

for exp_path in exp_paths:
    save_path = os.path.join(save_dir, exp_path) # куда сохраняем
    # пересоздаем если есть
    if os.path.exists(save_path):
        shutil.rmtree(save_path)
    os.makedirs(save_path)
    
    # подгружаем обученную модель
    model_cfg["pretrained_path"] = os.path.join(root_path, exp_path)
    model = cfg2model(model_cfg)
    
    # делаем словарь с плохими изображениями
    test_df = pd.read_excel(os.path.join(root_path, exp_path, "test", "full_results.xlsx"))
    bad_pictures = test_df[test_df[target_metric] < metric_value][["dir", "img"]]
    
    dir_dict = {}
    for dir_name in bad_pictures["dir"].unique():
        os.makedirs(os.path.join(save_path, dir_name))
        dir_dict[dir_name] = bad_pictures[bad_pictures["dir"] == dir_name]["img"].tolist()
    
    # создаем датасет, даталоадер с плохими изображениями
    testset = MultiImgMaskSet(
        log_name="test", root_path=data_path,
        dir_dict=dir_dict, max_subset_size=-1,
        bgr_trfm=A.Compose([]), fgr_trfm=A.Compose([]), trfm=A.Compose([]), preproc=A.Compose([]),
        device=torch.device(dataset_cfg["device"])
    )
    loader = DataLoader(testset, batch_size=1, shuffle=False)

    illustrate(model, loader, save_path)
    

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 60/60 [00:25<00:00,  2.33it/s]


In [ ]:
save_dir = "resized_bad_pictures" 

for exp_path in exp_paths:
    save_path = os.path.join(save_dir, exp_path) # куда сохраняем
    # пересоздаем если есть
    if os.path.exists(save_path):
        shutil.rmtree(save_path)
    os.makedirs(save_path)
    
    # подгружаем обученную модель
    model_cfg["pretrained_path"] = os.path.join(root_path, exp_path)
    model = cfg2model(model_cfg)
    
    # делаем словарь с плохими изображениями
    test_df = pd.read_excel(os.path.join(root_path, exp_path, "test", "full_results.xlsx"))
    bad_pictures = test_df[test_df[target_metric] < metric_value][["dir", "img"]]
    
    dir_dict = {}
    for dir_name in bad_pictures["dir"].unique():
        os.makedirs(os.path.join(save_path, dir_name))
        dir_dict[dir_name] = bad_pictures[bad_pictures["dir"] == dir_name]["img"].tolist()
    
    # создаем датасет, даталоадер с плохими изображениями
    testset = MultiImgMaskSet(
        log_name="test", root_path=data_path,
        dir_dict=dir_dict, max_subset_size=-1,
        bgr_trfm=A.Compose([]), fgr_trfm=A.Compose([]), trfm=A.Compose([]), preproc=A.Compose([]),
        device=torch.device(dataset_cfg["device"])
    )
    loader = DataLoader(testset, batch_size=1, shuffle=False)

    illustrate(model, loader, save_path)
    